- https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9

## Importings

In [1]:
import tweepy
import numpy as np
import yaml
import numpy as np
import os
import math

from datetime import datetime, timedelta
from time import sleep
from tqdm import tqdm
import pandas as pd

from unidecode import unidecode

In [2]:
with open('twitter_keys.yaml', 'r') as f:
    credentials = yaml.safe_load(f)

credentials

{'access_token': '1588313600942374912-IdEtMTjQ2p1Pj1ZWtQTFSxjcOk35EP',
 'access_token_secret': 'AHPh4D7bS7egiwYqRyMrXfIPXTmcQ9yxVEqYTHsTov9En',
 'Smoke Nova': {'api_key': 'wgr8tGnddtm58uVYItrekpfax',
  'api_key_secret': 'croFkMmPqnOIMagZklr8GRClpeYTZwqtsh7FsSN8GLg0iTyo2u',
  'bearer_token': 'AAAAAAAAAAAAAAAAAAAAADioiwEAAAAASqpNyScfLso0EfUwfV1aCAn4m60%3D6JzrW4Bno4LzCVXl7Ejn8PoDW0KF2dg5PngQ2bRQ1tGqlqvKGN'},
 'kul-thesis': {'api_key': 'zoby4xjcptFVVOGnbdjsWw4h2',
  'api_key_secret': '7D9HBgvIjN5TaHH7EBenrFq3D9e26jdOSwx7z0DXkQWjGZnHfI',
  'bearer_token': 'AAAAAAAAAAAAAAAAAAAAACWLlgEAAAAA%2BHeor7xDZWyWbRcm85krxfGAvQo%3DnmAkL2A0JB98DpyJq1ZmIABnOygfjmYFPeSfbslB3jYapj26kE'}}

In [3]:
TWITTER_API_ACCESS_TOKEN = credentials['access_token']
TWITTER_API_ACCESS_TOKEN_SECRET = credentials['access_token_secret']
TWITTER_API_KEY = credentials['kul-thesis']['api_key']
TWITTER_API_KEY_SECRET = credentials['kul-thesis']['api_key_secret']
TWITTER_BEARER_TOKEN = credentials['kul-thesis']['bearer_token']

In [5]:
auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_KEY_SECRET)
auth.set_access_token(TWITTER_API_ACCESS_TOKEN, TWITTER_API_ACCESS_TOKEN_SECRET)

tweepy_client = tweepy.Client(
    bearer_token=TWITTER_BEARER_TOKEN,
    consumer_key=TWITTER_API_KEY,
    consumer_secret=TWITTER_API_KEY_SECRET,
    access_token=TWITTER_API_ACCESS_TOKEN,
    access_token_secret=TWITTER_API_ACCESS_TOKEN_SECRET,
    wait_on_rate_limit=True
)

print("TWEEPY API client is successfully created...")

TWEEPY API client is successfully created...


## Sandbox

### Get all tweets from users

In [6]:
end_time = datetime(2023, 1, 30)
start_time = datetime(2022, 1, 1)

In [7]:
((end_time - start_time) / 10).days

39

In [15]:
def get_all_user_tweets(query, years=1, days_range=10, num_iter=39, end_time=datetime(2023, 1, 30)):
    """Get all tweets from a given user
    
    Example of query: from: mehdizana_ -is:retweet
    """
    
    # select time range
    # end_time = datetime.today()
    start_time = end_time - timedelta(days=days_range)
    
    all_tweets = []
    for i in tqdm(range(num_iter)):

        tweets = tweepy_client.search_all_tweets(
            query=query,
            tweet_fields=[
                "attachments",
                "author_id",
                "context_annotations",
                "conversation_id",
                "created_at",
                "edit_controls",
                "entities",
                "geo",
                "id",
                "in_reply_to_user_id",
                "lang",
                "public_metrics",
                "possibly_sensitive",
                "referenced_tweets",
                "reply_settings",
                "source",
                "text",
                "withheld"
            ],
            start_time=start_time,
            end_time=end_time,
            max_results=100
        )
        
        # include only valid tweets
        if tweets.meta['result_count'] > 0:
            all_tweets += tweets.data

            # alert when there is more than 100 tweets in the considered time range
            if tweets.meta['result_count'] > 100:
                print(f'There are more tweets between {start_time} and {end_time}')

        end_time -= timedelta(days=days_range)
        start_time -= timedelta(days=days_range)   

        sleep(1)
        
    all_tweets = pd.DataFrame(all_tweets)
        
    return all_tweets

In [16]:
df_hra = pd.read_excel('HRA Twitter Accounts.xlsx')

C:\Users\milto\AppData\Local\Temp/ipykernel_1292/1388474157.py:1: FutureWarning: Inferring datetime64[ns, UTC] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns, UTC])
  df_hra = pd.read_excel('HRA Twitter Accounts.xlsx')


In [18]:
saved_tweets = [i.replace('.csv', '') for i in os.listdir('tweets/')]
len(saved_tweets)

351

In [19]:
users = df_hra['screen_name'].dropna().values
len(users)

353

In [20]:
users = [i for i in users if i not in saved_tweets]
len(users)

0

In [14]:
for user in users:
    print(user)
    
    all_tweets = get_all_user_tweets(f'from: {user}')
    all_tweets.to_csv(f'tweets/{user}.csv', index=False)

meryemce74


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [01:13<00:00,  1.90s/it]


HaticeTopalogl


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [01:16<00:00,  1.97s/it]


profdrhakankar


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:56<00:00,  1.45s/it]


huseyingunturkn


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:52<00:00,  1.36s/it]


baskanibrahim06


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [01:00<00:00,  1.55s/it]
